In [1]:
from IPython.display import display
import os

# 上傳檔案（會跳出檔案選擇器）
from ipywidgets import FileUpload

upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

In [2]:
import os
from pathlib import Path

# 假設你只上傳了一個檔案
# Check if any file is uploaded and get the filename
if upload.value:
    # Get the first (and likely only) filename from the dictionary keys
    filename = list(upload.value.keys())[0]
    fileinfo = upload.value[filename] # Access the file info dictionary using the filename as key
else:
    print("No file uploaded.")
    # Handle the case where no file is uploaded, perhaps by exiting or prompting the user.
    # For this example, we'll assume a file was uploaded as per the traceback context.
    raise FileNotFoundError("No file was uploaded.")


# 顯示內容結構（除錯用）
print(fileinfo)

# 儲存 kaggle.json
# filename is already obtained above
content = fileinfo['content']

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

kaggle_json_path = kaggle_dir / "kaggle.json"
with open(kaggle_json_path, "wb") as f:
    f.write(content)

# 設定權限（Linux/macOS 建議）
os.chmod(kaggle_json_path, 0o600)

print(f"{filename} 已成功儲存至 {kaggle_json_path}")

{'metadata': {'name': 'kaggle.json', 'type': 'application/json', 'size': 64, 'lastModified': 1745215961986}, 'content': b'{"username":"suchiwen","key":"01a925cee9e9e9d232008524b0434fb9"}'}
kaggle.json 已成功儲存至 /root/.kaggle/kaggle.json


In [3]:
!kaggle datasets list -s cifar


ref                                                     title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
fedesoriano/cifar100                                    CIFAR-100 Python                                  168517809  2020-12-26 08:37:10.143000          12526        178  1.0              
pankrzysiu/cifar10-python                               CIFAR-10 Python                                   340613496  2018-01-27 13:42:40.967000          15066        255  0.75             
petitbonney/cifar10-image-recognition                   CIFAR-10                                         1007971063  2019-10-01 12:50:23.227000           2976         27  0.8235294        
valentynsichkar/cifar10-preprocessed                   

In [4]:
!pip install -U kaggle
!pip install --upgrade pandas
import os
import zipfile

# 建立 Kaggle 資料夾
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 下載 Dog Breed Identification 資料集
!kaggle competitions download -c dog-breed-identification --force
!unzip -oq dog-breed-identification.zip -d dog-breed-identification


import pandas as pd
import numpy as np

labels = pd.read_csv('dog-breed-identification/labels.csv')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 115.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cp: cannot stat 'kaggle.json': No such file or directory
 95% 659M/691M [00:05<00:01, 31.2MB/s]
100% 691M/691M [00:05<00:00, 122MB/s] 


In [15]:
# 安裝 transformers 套件
!pip install transformers

# 匯入 CosineAnnealingWarmup
from transformers import get_cosine_schedule_with_warmup

In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.cuda.amp import GradScaler, autocast
from collections import Counter
from transformers import get_cosine_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_epochs = 25
lr = 1e-4
num_workers = 2
val_ratio = 0.2
gamma = 2.0

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.cuda.amp import GradScaler, autocast
from collections import Counter
import math
import pandas as pd
from tqdm import tqdm

# 自訂特徵提取模型（Multi-Input）
class MultiInputVGG(nn.Module):
    def __init__(self, num_classes):
        super(MultiInputVGG, self).__init__()
        base = models.vgg16(pretrained=True)
        self.backbone = nn.Sequential(*list(base.features.children()))
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        self.feature_dim = 512  # vgg16 最後 conv 層輸出 channels

        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim * 3, 512),  # 合併 3 個局部特徵
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # 模擬輸入：耳、鼻、尾三個 crop 或 augment 區域
        # 預設全部從同張圖生成不同 augment
        feat1 = self.pool(self.backbone(x))
        feat2 = self.pool(self.backbone(x.flip(-1)))
        feat3 = self.pool(self.backbone(x[:, :, ::-1, :]))

        feat1 = feat1.view(x.size(0), -1)
        feat2 = feat2.view(x.size(0), -1)
        feat3 = feat3.view(x.size(0), -1)

        concat = torch.cat([feat1, feat2, feat3], dim=1)
        out = self.classifier(concat)
        return out


In [19]:
# CosineAnnealingWarmupLR
class CosineAnnealingWarmupLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_steps, total_steps, min_lr=0.0, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.min_lr = min_lr
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.warmup_steps:
            return [base_lr * self.last_epoch / self.warmup_steps for base_lr in self.base_lrs]
        else:
            progress = (self.last_epoch - self.warmup_steps) / (self.total_steps - self.warmup_steps)
            return [self.min_lr + (base_lr - self.min_lr) * 0.5 * (1 + math.cos(math.pi * progress)) for base_lr in self.base_lrs]


In [24]:
# ---------- 基本參數 ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_epochs = 25
lr = 1e-4
num_workers = 2
val_ratio = 0.2
gamma = 2.0 # Focal Loss gamma

In [25]:
# ---------- 資料預處理 ----------
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


In [27]:
# ---------- 自訂特徵提取模型（Multi-Input）----------
class MultiInputVGG(nn.Module):
    def __init__(self, num_classes):
        super(MultiInputVGG, self).__init__()
        # 加載預訓練的 VGG16 模型
        base = models.vgg16(pretrained=True)
        # 提取特徵提取層
        self.backbone = nn.Sequential(*list(base.features.children()))
        # 使用自適應平均池化確保輸出尺寸一致
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        # VGG16 features 的最後一層輸出通道數
        self.feature_dim = 512

        # 分類器，接收合併後的特徵
        self.classifier = nn.Sequential(
            # 由於模擬了 3 個輸入（或同一輸入的 3 個增強版本），特徵維度為 feature_dim * 3
            nn.Linear(self.feature_dim * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # 模擬處理同一張圖的不同區域或不同增強
        # 這裡簡單地對同一輸入進行不同的操作來模擬多輸入
        feat1 = self.pool(self.backbone(x))
        # 水平翻轉
        feat2 = self.pool(self.backbone(x.flip(-1)))
        # 垂直翻轉
        feat3 = self.pool(self.backbone(x.flip(-2))) # 使用 flip(-2) 進行垂直翻轉

        # 將池化後的特徵展平
        feat1 = feat1.view(x.size(0), -1)
        feat2 = feat2.view(x.size(0), -1)
        feat3 = feat3.view(x.size(0), -1)

        # 合併特徵
        concat = torch.cat([feat1, feat2, feat3], dim=1)
        # 通過分類器
        out = self.classifier(concat)
        return out


In [29]:
# ---------- Dataset ----------
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import random_split

all_breeds = sorted(labels['breed'].unique())
breed_to_idx = {breed: i for i, breed in enumerate(all_breeds)}
idx_to_breed = {i: breed for breed, i in breed_to_idx.items()} # For later use in submission

# Create a mapping from image id to class index
id_to_class_idx = labels.set_index('id')['breed'].map(breed_to_idx).to_dict()

# Custom Dataset for Dog Breed Identification
class DogBreedDataset(Dataset):
    def __init__(self, img_dir, id_to_class_idx, transform=None):
        self.img_dir = img_dir
        self.id_to_class_idx = id_to_class_idx
        # Get list of image files and filter out non-jpg files or files not in labels.csv
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg') and f[:-4] in id_to_class_idx]
        self.transform = transform


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_id = img_name[:-4] # Remove .jpg extension
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        label = self.id_to_class_idx[img_id]

        if self.transform:
            image = self.transform(image)

        return image, label

full_dataset = DogBreedDataset(img_dir="dog-breed-identification/train",
                               id_to_class_idx=id_to_class_idx,
                               transform=train_transform)

val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

# Split the dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
class DatasetWithTransform(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        # Retrieve the item from the underlying subset
        x, y = self.subset[index]
        # Apply the transform if it exists
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# Redo the Dataset creation and splitting to handle transforms correctly for train and validation
full_dataset_no_transform = DogBreedDataset(img_dir="dog-breed-identification/train",
                                             id_to_class_idx=id_to_class_idx,
                                             transform=None) # No transform initially

val_size = int(len(full_dataset_no_transform) * val_ratio)
train_size = len(full_dataset_no_transform) - val_size

# Split the dataset (subsets will return PIL Images)
train_subset, val_subset = random_split(full_dataset_no_transform, [train_size, val_size])

# Wrap the subsets to apply the specific transforms
train_dataset = DatasetWithTransform(train_subset, transform=train_transform)
val_dataset = DatasetWithTransform(val_subset, transform=val_transform)

# Assuming the class mapping from the full dataset is needed later
# You can access the original dataset from the subset if needed, or just use the id_to_class_idx directly
class_to_idx = breed_to_idx # Define class_to_idx here

In [35]:
# ---------- Sampler ----------
# Access the indices from the underlying subset stored in the wrapper
train_indices = train_dataset.subset.indices
# Assuming dataset.targets is accessible from the original dataset the subset was created from
# It is better to get the targets from the original full dataset before splitting
# Let's assume `full_dataset_no_transform.id_to_class_idx` contains the mapping from image ID to class index.
# We need to get the labels corresponding to the train_indices from the original data source.
# A more robust way is to get the labels directly from the train_subset.
train_labels = [full_dataset_no_transform.id_to_class_idx[full_dataset_no_transform.image_files[i][:-4]] for i in train_indices]


label_counts = Counter(train_labels)
class_sample_counts = [label_counts[i] for i in range(len(label_counts))]
sample_weights = [1.0 / class_sample_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [36]:
# ---------- Focal Loss ----------
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logpt = -self.ce(input, target)
        pt = torch.exp(logpt)
        loss = -((1 - pt) ** self.gamma) * logpt
        return loss.mean()

criterion = FocalLoss(gamma=gamma)

In [37]:
# ---------- 模型與優化器 ----------
model = MultiInputVGG(num_classes=len(label_counts)).to(device)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

steps_per_epoch = math.ceil(len(train_dataset) / batch_size)
total_steps = steps_per_epoch * num_epochs
warmup_steps = int(0.1 * total_steps)
scheduler = CosineAnnealingWarmupLR(optimizer, warmup_steps, total_steps)
scaler = GradScaler()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 73.1MB/s]
<ipython-input-37-417fb633f242>:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [39]:
# ---------- 訓練 ----------
best_acc = 0
train_losses, val_losses, val_accs = [], [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(imgs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    val_losses.append(avg_val_loss)
    val_accs.append(val_acc)
    if val_acc > best_acc:
        best_acc = val_acc

    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}")

print("\n📊 訓練完成統計：")
print(f"Average Train Loss: {sum(train_losses)/len(train_losses):.4f}")
print(f"Average Val Loss:   {sum(val_losses)/len(val_losses):.4f}")
print(f"Average Val Acc:    {sum(val_accs)/len(val_accs):.4f}")
print(f"Best Val Acc:       {best_acc:.4f}")



<ipython-input-39-29dbda14626e>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Epoch 1] Train Loss: 4.7115 | Val Loss: 4.7018 | Val Acc: 0.0113
[Epoch 2] Train Loss: 4.6899 | Val Loss: 4.6446 | Val Acc: 0.0259
[Epoch 3] Train Loss: 4.5782 | Val Loss: 4.2643 | Val Acc: 0.0558
[Epoch 4] Train Loss: 4.0426 | Val Loss: 3.0113 | Val Acc: 0.2216
[Epoch 5] Train Loss: 3.4735 | Val Loss: 2.3312 | Val Acc: 0.3307
[Epoch 6] Train Loss: 3.1162 | Val Loss: 1.9050 | Val Acc: 0.4031
[Epoch 7] Train Loss: 2.8643 | Val Loss: 1.6060 | Val Acc: 0.4628
[Epoch 8] Train Loss: 2.6764 | Val Loss: 1.3943 | Val Acc: 0.5024
[Epoch 9] Train Loss: 2.4807 | Val Loss: 1.1685 | Val Acc: 0.5431
[Epoch 10] Train Loss: 2.2532 | Val Loss: 1.0623 | Val Acc: 0.5592
[Epoch 11] Train Loss: 2.0947 | Val Loss: 0.9800 | Val Acc: 0.5690
[Epoch 12] Train Loss: 1.9559 | Val Loss: 0.8736 | Val Acc: 0.6008
[Epoch 13] Train Loss: 1.8727 | Val Loss: 0.8461 | Val Acc: 0.5954
[Epoch 14] Train Loss: 1.7899 | Val Loss: 0.7109 | Val Acc: 0.6282
[Epoch 15] Train Loss: 1.6420 | Val Loss: 0.6826 | Val Acc: 0.6370
[Epo

In [44]:
# ---------- 預測 ----------
model.eval()
predictions = []
image_names_list = []

with torch.no_grad():
    for image_names, images in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()  # Softmax 機率
        predictions.extend(probs)
        image_names_list.extend(image_names)

# ---------- 產生 submission.csv ----------
# 以 class_to_idx 建立 idx_to_class
idx_to_class = {v: k for k, v in class_to_idx.items()}
# 照 class index 排序，確保欄位順序一致
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]

submission_df = pd.DataFrame(predictions, columns=class_names)
submission_df.insert(0, "id", [name for name in image_names_list])
submission_df["id"] = submission_df["id"].str.replace(".jpg", "", regex=False)
submission_df.to_csv("submission.csv", index=False)
print("✅ 已產出符合格式的 submission.csv！")


✅ 已產出符合格式的 submission.csv！
